# Capstone Projekt Rossmann

# Feature Engineering

## Functions needed for testing

### Test models with test and tain data. Test includes the last 8 weeks from each store

In [401]:
## Test models with test and tain data. Test includes the last 8 weeks from each store

def testModelsTestSplit8W(df, scaler):
	train_data = []
	test_data = []

	# Group by store and split into training and test data
	amount_test_weeks = 8
	for store_id, group in df.groupby('Store'):
		train_data.append(group[: -amount_test_weeks])
		test_data.append(group[-amount_test_weeks:])

	# Combine the list entries to one dataframe
	train_df = pd.concat(train_data)
	test_df = pd.concat(test_data)

	X_train = train_df.drop(columns=['Future_Sales'])
	y_train = train_df['Future_Sales']
	X_test = test_df.drop(columns=['Future_Sales'])
	y_test = test_df['Future_Sales']

	# Scaling of the data
	if scaler:
		X_train = scaler.fit_transform(X_train)
		X_test = scaler.transform(X_test)

	def adj_r2_score(model, X, y):
		n = X.shape[0]
		p = X.shape[1]
		r2 = r2_score(y, model.predict(X))
		return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

	# Defining the models to test
	models = [
		('LinearRegression', LinearRegression(n_jobs=-1)),
		#('RidgeRegression', Ridge(random_state=42)),
		#('LassoRegression', Lasso(random_state=42)),
		#('DecisionTreeRegressor', DecisionTreeRegressor(random_state=42)),
		#('RandomForestRegressor', RandomForestRegressor(n_jobs=-1, max_depth=10, random_state=42, n_estimators=100)),
		#('SVR', SVR()),
		#('KNN', KNeighborsRegressor())
	]

	results = []
	# Train models and calculate metrics
	for name, model in models:
		model.fit(X_train, y_train)
		y_train_pred = model.predict(X_train)
		y_test_pred = model.predict(X_test)

		results.append({
			'Model': name,
			'RMSE_Train': sqrt(mse(y_train, y_train_pred)),
			'MAE_Train': mae(y_train, y_train_pred),
			'R2_Train': r2_score(y_train, y_train_pred),
			'Adj_R2_Train': adj_r2_score(model, X_train, y_train),
			'RMSE_Test': sqrt(mse(y_test, y_test_pred)),
			'MAE_Test': mae(y_test, y_test_pred),
			'R2_Test': r2_score(y_test, y_test_pred),
			'Adj_R2_Test': adj_r2_score(model, X_test, y_test)
		})
		#print last result
		print(results[-1])

	results_df = pd.DataFrame(results)
	return results_df

### Creates x splits in test and train where the last 8 weeks of each store are included in the respective test split and the splits are distributed evenly using gap

In [402]:
#Creates x splits in test and train where the last 8 weeks of each store are included in the respective test split and the splits are distributed
# evenly using gap

def testModelsCV8W(df, scaler):

    n_splits = 5
    window_size = 8
    total_weeks =109
    train_size = window_size / 0.2
    gap = int((total_weeks - window_size - train_size) // (n_splits))

    results = []

    for split in range(n_splits):
        train_data = []
        test_data = []

        for store_id, group in df.groupby('Store'):
            # calculate start and end index for test data
            if split == 0:
                test_start_index = -window_size
                test_df_store = group[test_start_index:]  # Kein Endindex für den ersten Split
            else:
                test_start_index = -(window_size + gap * split)
                test_end_index = test_start_index + window_size
                test_df_store = group[test_start_index:test_end_index]
                print("test:", test_df_store.shape, "Test Start Index:", test_start_index, "Test End Index:", test_end_index)
            
            train_start_index = -int(-test_start_index + gap + train_size)
            train_df_store = group[train_start_index:test_start_index]
            print("Train:", train_df_store.shape, "Train Start Index:", train_start_index, "Train End Index:", test_start_index)
            # Check if test set contains data
            if not test_df_store.empty:
                train_data.append(train_df_store)
                test_data.append(test_df_store)
            else:
                print(f"Store {store_id} has not enough data for splitting {split}")

        # Combine the list entries to one dataframe
        train_df_combined = pd.concat(train_data)
        test_df_combined = pd.concat(test_data)

        # Create feature and target data frames
        X_train = train_df_combined.drop(columns=['Future_Sales'])
        y_train = train_df_combined['Future_Sales']
        X_test = test_df_combined.drop(columns=['Future_Sales'])
        y_test = test_df_combined['Future_Sales']

        # Scaling of the data
        if scaler:
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        def adj_r2_score(model, X, y):
            n = X.shape[0]
            p = X.shape[1]
            r2 = r2_score(y, model.predict(X))
            return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

    	# Defining the models to test
        models = [
            ('LinearRegression', LinearRegression(n_jobs=-1)),
            #('RidgeRegression', Ridge(random_state=42)),
            #('LassoRegression', Lasso(random_state=42)),
            #('DecisionTreeRegressor', DecisionTreeRegressor(random_state=42)),
            #('RandomForestRegressor', RandomForestRegressor(n_jobs=-1, max_depth=10, random_state=42, n_estimators=100)),
            #('SVR', SVR()),
            #('KNN', KNeighborsRegressor())
        ]
        
        # Train models and calculate metrics
        for name, model in models:
            model.fit(X_train, y_train)
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
        
            results.append({
                'Model': name,
                'RMSE_Train': sqrt(mse(y_train, y_train_pred)),
                'MAE_Train': mae(y_train, y_train_pred),
                'R2_Train': r2_score(y_train, y_train_pred),
                'Adj_R2_Train': adj_r2_score(model, X_train, y_train),
                'RMSE_Test': sqrt(mse(y_test, y_test_pred)),
                'MAE_Test': mae(y_test, y_test_pred),
                'R2_Test': r2_score(y_test, y_test_pred),
                'Adj_R2_Test': adj_r2_score(model, X_test, y_test)
            })
            #print last result
            print(results[-1])

    results_df = pd.DataFrame(results)

    # calculate mean of all splits
    model_list = results_df['Model'].unique()
    # create resulte_mean_df
    resulte_mean_df = pd.DataFrame(columns=results_df.columns)
    # iterate over model_list
    for model in model_list:
        # get mean of each model
        mean = results_df[results_df['Model'] == model].mean(numeric_only=True)
        mean['Model'] = model
        # append mean to resulte_mean_df
        resulte_mean_df = pd.concat([resulte_mean_df, pd.DataFrame([mean], columns=results_df.columns)], ignore_index=True)

    return results_df, resulte_mean_df


## Feature Engineering

In [403]:
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from pandas.api.types import infer_dtype

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score
from math import sqrt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import RobustScaler

pd.set_option('display.max_columns', None)

In [404]:
df = pd.read_csv('weekly_sales_with_store_info.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [405]:
print(df.info())
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150525 entries, 0 to 150524
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      150525 non-null  int64         
 1   Date                       150525 non-null  datetime64[ns]
 2   CW                         150525 non-null  int64         
 3   Month                      150525 non-null  int64         
 4   Year                       150525 non-null  int64         
 5   DayOfWeek                  150525 non-null  int64         
 6   Sales                      150525 non-null  int64         
 7   SalesPerCustomer           145809 non-null  float64       
 8   SalesPerOpenDay            145815 non-null  float64       
 9   Customers                  150525 non-null  int64         
 10  CustomersPerOpenDay        145815 non-null  float64       
 11  Open                       150525 non-null  int64   

,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,IsCompetition,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2Member,Promo2Active
31189,232,2013-02-03,5,2,2013,6,22049,11.018991,3674.833333,2001,333.500000,6,0,0,0,0,1,1,0,c,c,13570.0,5.0,2010.0,1,1,10.0,2013.0,"Mar,Jun,Sept,Dec",0,0
91352,677,2014-10-12,41,10,2014,6,0,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,d,a,1740.0,6.0,2014.0,1,1,45.0,2009.0,"Feb,May,Aug,Nov",1,0
138301,1025,2014-03-09,10,3,2014,6,43523,8.675105,8704.600000,5017,1003.400000,5,5,1,0,0,0,0,0,a,a,720.0,11.0,2009.0,1,0,NaN,NaN,NaN,0,0
53658,398,2014-03-23,12,3,2014,6,35354,9.154324,5892.333333,3862,643.666667,6,5,1,0,0,0,0,0,c,c,1540.0,NaN,NaN,0,1,1.0,2012.0,"Jan,Apr,Jul,Oct",1,0
84709,628,2014-03-30,13,3,2014,6,28931,7.274579,4821.833333,3977,662.833333,6,0,0,0,0,0,0,0,a,c,2180.0,NaN,NaN,0,0,NaN,NaN,NaN,0,0


### Handle Missing Values

In [406]:
df.isna().sum()

Store                            0
Date                             0
CW                               0
Month                            0
Year                             0
DayOfWeek                        0
Sales                            0
SalesPerCustomer              4716
SalesPerOpenDay               4710
Customers                        0
CustomersPerOpenDay           4710
Open                             0
Promo                            0
IsPromo                          0
StateHoliday                     0
IsStateHoliday                   0
SchoolHoliday                    0
IsSchoolHoliday                  0
NumStateHoliday                  0
StoreType                        0
Assortment                       0
CompetitionDistance            405
CompetitionOpenSinceMonth    47790
CompetitionOpenSinceYear     47790
IsCompetition                    0
Promo2                           0
Promo2SinceWeek              73440
Promo2SinceYear              73440
PromoInterval       

#### SalesPerCustomer, SalesPerOpenday, CustomersPerOpenday

In [407]:
# As the store were closed, we can fill the nans with 0

# fill nans with 0 for listed columns
columns_to_fill = ['SalesPerCustomer', 'SalesPerOpenDay', 'CustomersPerOpenDay']
df_nans_handeled = df.fillna({col: 0 for col in columns_to_fill})
df_nans_handeled


,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,IsCompetition,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2Member,Promo2Active
0,1,2013-01-06,1,1,2013,6,19340,7.736000,4835.000000,2500,625.000000,4,0,0,a,1,6,1,1,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
1,1,2013-01-13,2,1,2013,6,32952,8.410413,5492.000000,3918,653.000000,6,5,1,0,0,5,1,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
2,1,2013-01-20,3,1,2013,6,25978,7.602575,4329.666667,3417,569.500000,6,0,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
3,1,2013-01-27,4,1,2013,6,33071,8.563180,5511.833333,3862,643.666667,6,5,1,0,0,0,0,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
4,1,2013-02-03,5,2,2013,6,28693,8.057568,4782.166667,3561,593.500000,6,0,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,1,0,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,2015-07-05,27,7,2015,6,48130,16.140174,8021.666667,2982,497.000000,6,5,1,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0
150521,1115,2015-07-12,28,7,2015,6,36233,14.315685,6038.833333,2531,421.833333,6,0,0,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0
150522,1115,2015-07-19,29,7,2015,6,45927,15.023553,7654.500000,3057,509.500000,6,5,1,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0
150523,1115,2015-07-26,30,7,2015,6,35362,14.122204,5893.666667,2504,417.333333,6,0,0,0,0,0,0,0,d,c,5350.0,NaN,NaN,0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,0


In [408]:
df_nans_handeled.isna().sum()

Store                            0
Date                             0
CW                               0
Month                            0
Year                             0
DayOfWeek                        0
Sales                            0
SalesPerCustomer                 0
SalesPerOpenDay                  0
Customers                        0
CustomersPerOpenDay              0
Open                             0
Promo                            0
IsPromo                          0
StateHoliday                     0
IsStateHoliday                   0
SchoolHoliday                    0
IsSchoolHoliday                  0
NumStateHoliday                  0
StoreType                        0
Assortment                       0
CompetitionDistance            405
CompetitionOpenSinceMonth    47790
CompetitionOpenSinceYear     47790
IsCompetition                    0
Promo2                           0
Promo2SinceWeek              73440
Promo2SinceYear              73440
PromoInterval       

#### CompetitionDistance

In [409]:
# Stores with no CompetitionDistance information
print("Stores with no CompetitionDistance information:", df_nans_handeled[(df_nans_handeled['CompetitionDistance'].isna())]['Store'].unique())

print("StoreType of store 291", df_nans_handeled[(df_nans_handeled['Store'] == 291)]['StoreType'].unique())
print("StoreType of store 622", df_nans_handeled[(df_nans_handeled['Store'] == 622)]['StoreType'].unique())
print("StoreType of store 879", df_nans_handeled[(df_nans_handeled['Store'] == 879)]['StoreType'].unique())

Stores with no CompetitionDistance information: [291 622 879]
StoreType of store 291 ['d']
StoreType of store 622 ['a']
StoreType of store 879 ['d']


In [410]:
# As store 291, 622 and 879 have no CompetitionDistance information, we can fill them with the median value of the column

# median competition distance for store type a
median_competition_distance_a = df_nans_handeled[(df_nans_handeled['StoreType'] == 'a')]['CompetitionDistance'].median()
# median competition distance for store type d
median_competition_distance_d = df_nans_handeled[(df_nans_handeled['StoreType'] == 'd')]['CompetitionDistance'].median()

# fill nans for storetype a with median_competition_distance_a
df_nans_handeled.loc[(df_nans_handeled['Store'] == 291), 'CompetitionDistance'] = median_competition_distance_a
# fill nans for storetype d with median_competition_distance_d
df_nans_handeled.loc[(df_nans_handeled['Store'] == 622), 'CompetitionDistance'] = median_competition_distance_d
# fill nans for storetype d with median_competition_distance_d
df_nans_handeled.loc[(df_nans_handeled['Store'] == 879), 'CompetitionDistance'] = median_competition_distance_d



In [411]:
df_nans_handeled.isna().sum()

Store                            0
Date                             0
CW                               0
Month                            0
Year                             0
DayOfWeek                        0
Sales                            0
SalesPerCustomer                 0
SalesPerOpenDay                  0
Customers                        0
CustomersPerOpenDay              0
Open                             0
Promo                            0
IsPromo                          0
StateHoliday                     0
IsStateHoliday                   0
SchoolHoliday                    0
IsSchoolHoliday                  0
NumStateHoliday                  0
StoreType                        0
Assortment                       0
CompetitionDistance              0
CompetitionOpenSinceMonth    47790
CompetitionOpenSinceYear     47790
IsCompetition                    0
Promo2                           0
Promo2SinceWeek              73440
Promo2SinceYear              73440
PromoInterval       

#### CompetitionOpenSinceMonth, CompetitionOpenSinceYear

In [412]:
# CompetitionOpenSinceMonth and CompetitionOpenSinceYear can be deleted as they are reflected in IsCompetition
df_nans_handeled = df_nans_handeled.drop(columns=['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'])

In [413]:
df_nans_handeled.isna().sum()

Store                      0
Date                       0
CW                         0
Month                      0
Year                       0
DayOfWeek                  0
Sales                      0
SalesPerCustomer           0
SalesPerOpenDay            0
Customers                  0
CustomersPerOpenDay        0
Open                       0
Promo                      0
IsPromo                    0
StateHoliday               0
IsStateHoliday             0
SchoolHoliday              0
IsSchoolHoliday            0
NumStateHoliday            0
StoreType                  0
Assortment                 0
CompetitionDistance        0
IsCompetition              0
Promo2                     0
Promo2SinceWeek        73440
Promo2SinceYear        73440
PromoInterval          73440
Promo2Member               0
Promo2Active               0
dtype: int64

#### Promo2SinceWeek, Promo2SinceYear

In [414]:
# Promo2SinceWeek and Promo2SinceYear can be deleted as they are reflected in Promo2Member
df_nans_handeled = df_nans_handeled.drop(columns=['Promo2SinceWeek', 'Promo2SinceYear'])

In [415]:
df_nans_handeled.isna().sum()

Store                      0
Date                       0
CW                         0
Month                      0
Year                       0
DayOfWeek                  0
Sales                      0
SalesPerCustomer           0
SalesPerOpenDay            0
Customers                  0
CustomersPerOpenDay        0
Open                       0
Promo                      0
IsPromo                    0
StateHoliday               0
IsStateHoliday             0
SchoolHoliday              0
IsSchoolHoliday            0
NumStateHoliday            0
StoreType                  0
Assortment                 0
CompetitionDistance        0
IsCompetition              0
Promo2                     0
PromoInterval          73440
Promo2Member               0
Promo2Active               0
dtype: int64

#### PromoInterval

In [416]:
df_nans_handeled[(df_nans_handeled['Promo2'] == 1) & (df_nans_handeled['PromoInterval'].isna())]

,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,IsCompetition,Promo2,PromoInterval,Promo2Member,Promo2Active


In [417]:
# As if the store is not participating in Promo2, PromoInterval is 0, we can fill the nans with 0
df_nans_handeled['PromoInterval'] = df_nans_handeled['PromoInterval'].fillna(0)

In [418]:
df_nans_handeled.isna().sum()

Store                  0
Date                   0
CW                     0
Month                  0
Year                   0
DayOfWeek              0
Sales                  0
SalesPerCustomer       0
SalesPerOpenDay        0
Customers              0
CustomersPerOpenDay    0
Open                   0
Promo                  0
IsPromo                0
StateHoliday           0
IsStateHoliday         0
SchoolHoliday          0
IsSchoolHoliday        0
NumStateHoliday        0
StoreType              0
Assortment             0
CompetitionDistance    0
IsCompetition          0
Promo2                 0
PromoInterval          0
Promo2Member           0
Promo2Active           0
dtype: int64

### Remove not needed Features

In [419]:
# Store is just an ID
#df_nans_handeled = df_nans_handeled.drop(columns=['Store'])

# Date is an object and is reflected by CW, Month and Year
df_nans_handeled = df_nans_handeled.drop(columns=['Date'])

# DayOfWeek is not relevant in weekly data
df_nans_handeled = df_nans_handeled.drop(columns=['DayOfWeek'])

df_nans_handeled

,Store,CW,Month,Year,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,IsCompetition,Promo2,PromoInterval,Promo2Member,Promo2Active
0,1,1,1,2013,19340,7.736000,4835.000000,2500,625.000000,4,0,0,a,1,6,1,1,c,a,1270.0,1,0,0,0,0
1,1,2,1,2013,32952,8.410413,5492.000000,3918,653.000000,6,5,1,0,0,5,1,0,c,a,1270.0,1,0,0,0,0
2,1,3,1,2013,25978,7.602575,4329.666667,3417,569.500000,6,0,0,0,0,0,0,0,c,a,1270.0,1,0,0,0,0
3,1,4,1,2013,33071,8.563180,5511.833333,3862,643.666667,6,5,1,0,0,0,0,0,c,a,1270.0,1,0,0,0,0
4,1,5,2,2013,28693,8.057568,4782.166667,3561,593.500000,6,0,0,0,0,0,0,0,c,a,1270.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,27,7,2015,48130,16.140174,8021.666667,2982,497.000000,6,5,1,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0
150521,1115,28,7,2015,36233,14.315685,6038.833333,2531,421.833333,6,0,0,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0
150522,1115,29,7,2015,45927,15.023553,7654.500000,3057,509.500000,6,5,1,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0
150523,1115,30,7,2015,35362,14.122204,5893.666667,2504,417.333333,6,0,0,0,0,0,0,0,d,c,5350.0,0,1,"Mar,Jun,Sept,Dec",1,0


In [420]:
#df_cleaned

### Categorical Feature Encoding

In [421]:
df_nans_handeled.select_dtypes(include='object').columns

Index(['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval'], dtype='object')

In [422]:
# check if a column contains mixed data typesrom pandas.api.types import infer_dtype

for col in ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']:
    dtype = infer_dtype(df_nans_handeled[col])
    print(f"Data type of {col}: {dtype}")


Data type of StateHoliday: string
Data type of StoreType: string
Data type of Assortment: string
Data type of PromoInterval: mixed-integer


In [423]:
# Convert mixed columns
cols_to_convert = ['PromoInterval']
df_nans_handeled[cols_to_convert] = df_nans_handeled[cols_to_convert].astype(str)

In [424]:
from sklearn.preprocessing import OneHotEncoder
# handle_unknown='ignore': um Fehler zu vermeiden, wenn die Trainingsdaten Klassen/Kategorien enthalten, die nicht in den Trainingsdaten vertreten sind
# sparse=False: stellt sicher, dass die kodierten Spalten als NumPy-Array zurückgegeben werden (anstelle einer Sparse-Matrix).
OneHotEnc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# Es dürfen nur die Kategorialen Spalten üergeben werden, nicht das ganze df
encoded_array = OneHotEnc.fit_transform(df_nans_handeled[['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']])
tmp_cat = pd.DataFrame(encoded_array, columns=OneHotEnc.get_feature_names_out(), index=df_nans_handeled.index)
df_nans_handeled_cat = pd.concat([df_nans_handeled.select_dtypes(include=['number']), tmp_cat], axis=1)
df_nans_handeled_cat

,Store,CW,Month,Year,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,CompetitionDistance,IsCompetition,Promo2,Promo2Member,Promo2Active,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
0,1,1,1,2013,19340,7.736000,4835.000000,2500,625.000000,4,0,0,1,6,1,1,1270.0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,1,2013,32952,8.410413,5492.000000,3918,653.000000,6,5,1,0,5,1,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,1,2013,25978,7.602575,4329.666667,3417,569.500000,6,0,0,0,0,0,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,4,1,2013,33071,8.563180,5511.833333,3862,643.666667,6,5,1,0,0,0,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,5,2,2013,28693,8.057568,4782.166667,3561,593.500000,6,0,0,0,0,0,0,1270.0,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150520,1115,27,7,2015,48130,16.140174,8021.666667,2982,497.000000,6,5,1,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
150521,1115,28,7,2015,36233,14.315685,6038.833333,2531,421.833333,6,0,0,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
150522,1115,29,7,2015,45927,15.023553,7654.500000,3057,509.500000,6,5,1,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
150523,1115,30,7,2015,35362,14.122204,5893.666667,2504,417.333333,6,0,0,0,0,0,0,5350.0,0,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [425]:
# Lag-Features für diese Spalten erstellen
lag_columns = ['Sales', 'SalesPerCustomer', 'SalesPerOpenDay', 'Customers', 'CustomersPerOpenDay']
n_lags = 8  # Anzahl der zu erstellenden Lag-Features
n_periods_for_ma = 4  # Anzahl der Perioden für den gleitenden Durchschnitt
n_periods_for_ma2 = 6  # Anzahl der Perioden für den gleitenden Durchschnitt
n_periods_for_ma3 = 8  # Anzahl der Perioden für den gleitenden Durchschnitt

lag_features = []
for col in lag_columns:
    store_groups = df_nans_handeled_cat.groupby('Store')[col]
    for lag in range(1, n_lags + 1):
        lag_col_name = f'{col}_Lag_{lag}'
        # Erstellen des Lag-Features
        lag_feature = store_groups.shift(lag).rename(lag_col_name)
        lag_features.append(lag_feature)

        # Berechnen des gleitenden Durchschnitts für das Lag-Feature
        ma_col_name = f'{lag_col_name}_MA_{n_periods_for_ma}'
        ma_feature = lag_feature.rolling(window=n_periods_for_ma).mean().rename(ma_col_name)
        lag_features.append(ma_feature)
        
        ma2_col_name = f'{lag_col_name}_MA_{n_periods_for_ma2}'
        ma2_feature = lag_feature.rolling(window=n_periods_for_ma2).mean().rename(ma2_col_name)
        lag_features.append(ma2_feature)
        
        ma3_col_name = f'{lag_col_name}_MA_{n_periods_for_ma3}'
        ma3_feature = lag_feature.rolling(window=n_periods_for_ma3).mean().rename(ma3_col_name)
        lag_features.append(ma3_feature)

# Fügen Sie alle berechneten Features hinzu
features_df = pd.concat(lag_features, axis=1)
df_nans_handeled_cat = pd.concat([df_nans_handeled_cat, features_df], axis=1)

# Fügen Sie die zukünftigen Verkäufe ein
future_sales = df_nans_handeled_cat.groupby('Store')['Sales'].shift(-8).rename('Future_Sales')
df_nans_handeled_cat = pd.concat([df_nans_handeled_cat, future_sales], axis=1)

# Entfernen von Zeilen mit NaN-Werten, die durch das Verschieben entstanden sind, und Entfernen der Original-Spalten
df_nans_handeled_cat = df_nans_handeled_cat.dropna().drop(columns=lag_columns)
df_nans_handeled_cat


,Store,CW,Month,Year,Open,Promo,IsPromo,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,CompetitionDistance,IsCompetition,Promo2,Promo2Member,Promo2Active,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",Sales_Lag_1,Sales_Lag_1_MA_4,Sales_Lag_1_MA_6,Sales_Lag_1_MA_8,Sales_Lag_2,Sales_Lag_2_MA_4,Sales_Lag_2_MA_6,Sales_Lag_2_MA_8,Sales_Lag_3,Sales_Lag_3_MA_4,Sales_Lag_3_MA_6,Sales_Lag_3_MA_8,Sales_Lag_4,Sales_Lag_4_MA_4,Sales_Lag_4_MA_6,Sales_Lag_4_MA_8,Sales_Lag_5,Sales_Lag_5_MA_4,Sales_Lag_5_MA_6,Sales_Lag_5_MA_8,Sales_Lag_6,Sales_Lag_6_MA_4,Sales_Lag_6_MA_6,Sales_Lag_6_MA_8,Sales_Lag_7,Sales_Lag_7_MA_4,Sales_Lag_7_MA_6,Sales_Lag_7_MA_8,Sales_Lag_8,Sales_Lag_8_MA_4,Sales_Lag_8_MA_6,Sales_Lag_8_MA_8,SalesPerCustomer_Lag_1,SalesPerCustomer_Lag_1_MA_4,SalesPerCustomer_Lag_1_MA_6,SalesPerCustomer_Lag_1_MA_8,SalesPerCustomer_Lag_2,SalesPerCustomer_Lag_2_MA_4,SalesPerCustomer_Lag_2_MA_6,SalesPerCustomer_Lag_2_MA_8,SalesPerCustomer_Lag_3,SalesPerCustomer_Lag_3_MA_4,SalesPerCustomer_Lag_3_MA_6,SalesPerCustomer_Lag_3_MA_8,SalesPerCustomer_Lag_4,SalesPerCustomer_Lag_4_MA_4,SalesPerCustomer_Lag_4_MA_6,SalesPerCustomer_Lag_4_MA_8,SalesPerCustomer_Lag_5,SalesPerCustomer_Lag_5_MA_4,SalesPerCustomer_Lag_5_MA_6,SalesPerCustomer_Lag_5_MA_8,SalesPerCustomer_Lag_6,SalesPerCustomer_Lag_6_MA_4,SalesPerCustomer_Lag_6_MA_6,SalesPerCustomer_Lag_6_MA_8,SalesPerCustomer_Lag_7,SalesPerCustomer_Lag_7_MA_4,SalesPerCustomer_Lag_7_MA_6,SalesPerCustomer_Lag_7_MA_8,SalesPerCustomer_Lag_8,SalesPerCustomer_Lag_8_MA_4,SalesPerCustomer_Lag_8_MA_6,SalesPerCustomer_Lag_8_MA_8,SalesPerOpenDay_Lag_1,SalesPerOpenDay_Lag_1_MA_4,SalesPerOpenDay_Lag_1_MA_6,SalesPerOpenDay_Lag_1_MA_8,SalesPerOpenDay_Lag_2,SalesPerOpenDay_Lag_2_MA_4,SalesPerOpenDay_Lag_2_MA_6,SalesPerOpenDay_Lag_2_MA_8,SalesPerOpenDay_Lag_3,SalesPerOpenDay_Lag_3_MA_4,SalesPerOpenDay_Lag_3_MA_6,SalesPerOpenDay_Lag_3_MA_8,SalesPerOpenDay_Lag_4,SalesPerOpenDay_Lag_4_MA_4,SalesPerOpenDay_Lag_4_MA_6,SalesPerOpenDay_Lag_4_MA_8,SalesPerOpenDay_Lag_5,SalesPerOpenDay_Lag_5_MA_4,SalesPerOpenDay_Lag_5_MA_6,SalesPerOpenDay_Lag_5_MA_8,SalesPerOpenDay_Lag_6,SalesPerOpenDay_Lag_6_MA_4,SalesPerOpenDay_Lag_6_MA_6,SalesPerOpenDay_Lag_6_MA_8,SalesPerOpenDay_Lag_7,SalesPerOpenDay_Lag_7_MA_4,SalesPerOpenDay_Lag_7_MA_6,SalesPerOpenDay_Lag_7_MA_8,SalesPerOpenDay_Lag_8,SalesPerOpenDay_Lag_8_MA_4,SalesPerOpenDay_Lag_8_MA_6,SalesPerOpenDay_Lag_8_MA_8,Customers_Lag_1,Customers_Lag_1_MA_4,Customers_Lag_1_MA_6,Customers_Lag_1_MA_8,Customers_Lag_2,Customers_Lag_2_MA_4,Customers_Lag_2_MA_6,Customers_Lag_2_MA_8,Customers_Lag_3,Customers_Lag_3_MA_4,Customers_Lag_3_MA_6,Customers_Lag_3_MA_8,Customers_Lag_4,Customers_Lag_4_MA_4,Customers_Lag_4_MA_6,Customers_Lag_4_MA_8,Customers_Lag_5,Customers_Lag_5_MA_4,Customers_Lag_5_MA_6,Customers_Lag_5_MA_8,Customers_Lag_6,Customers_Lag_6_MA_4,Customers_Lag_6_MA_6,Customers_Lag_6_MA_8,Customers_Lag_7,Customers_Lag_7_MA_4,Customers_Lag_7_MA_6,Customers_Lag_7_MA_8,Customers_Lag_8,Customers_Lag_8_MA_4,Customers_Lag_8_MA_6,Customers_Lag_8_MA_8,CustomersPerOpenDay_Lag_1,CustomersPerOpenDay_Lag_1_MA_4,CustomersPerOpenDay_Lag_1_MA_6,CustomersPerOpenDay_Lag_1_MA_8,CustomersPerOpenDay_Lag_2,CustomersPerOpenDay_Lag_2_MA_4,CustomersPerOpenDay_Lag_2_MA_6,CustomersPerOpenDay_Lag_2_MA_8,CustomersPerOpenDay_Lag_3,CustomersPerOpenDay_Lag_3_MA_4,CustomersPerOpenDay_Lag_3_MA_6,CustomersPerOpenDay_Lag_3_MA_8,CustomersPerOpenDay_Lag_4,CustomersPerOpenDay_Lag_4_MA_4,CustomersPerOpenDay_Lag_4_MA_6,CustomersPerOpenDay_Lag_4_MA_8,CustomersPerOpenDay_Lag_5,CustomersPerOpenDay_Lag_5_MA_4,CustomersPerOpenDay_Lag_5_MA_6,CustomersPerOpenDay_Lag_5_MA_8,CustomersPerOpenDay_Lag_6,CustomersPerOpenDay_Lag_6_MA_4,CustomersPerOpenDay_Lag_6_MA_6,CustomersPerOpenDay_Lag_6_MA_8,CustomersPerOpenDay_Lag_7,CustomersPerOpen

### New additional Features

#### Ploynominal Features

In [426]:
# Before
testModelsTestSplit8W(df_nans_handeled_cat, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9348.624262728845, 'MAE_Train': 6448.18666177155, 'R2_Train': 0.7367209231767318, 'Adj_R2_Train': 0.7362865518161379, 'RMSE_Test': 6029.131809472689, 'MAE_Test': 4495.405879127806, 'R2_Test': 0.8600745236580398, 'Adj_R2_Test': 0.8570124514786959}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9348.624263,6448.186662,0.736721,0.736287,6029.131809,4495.405879,0.860075,0.857012


In [427]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2)
features = ['Open', 'Promo', 'IsPromo', 'IsStateHoliday', 'SchoolHoliday', 'IsSchoolHoliday', 'CompetitionDistance', 'IsCompetition', 'Promo2Member', 'Assortment_a', 'Assortment_b', 'Assortment_c']

feat_array = pf.fit_transform(df_nans_handeled_cat[features])
# Generieren der Namen für die neuen Features
feature_names = pf.get_feature_names_out(input_features=features)
# Umwandlung in einen DataFrame
poly_features_df = pd.DataFrame(feat_array, columns=feature_names, index=df_nans_handeled_cat.index)
# Entfernen der '1'-Spalte, wenn nicht benötigt (optional)
poly_features_df = poly_features_df.drop('1', axis=1)
# Zusammenführen der polynomialen Features mit dem ursprünglichen DataFrame
df_nans_handeled_cat_poly = pd.concat([df_nans_handeled_cat, poly_features_df], axis=1)
testModelsTestSplit8W(df_nans_handeled_cat_poly, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9196.055248192568, 'MAE_Train': 6324.875376361263, 'R2_Train': 0.745244200308042, 'Adj_R2_Train': 0.7446253585255644, 'RMSE_Test': 6753.577811387857, 'MAE_Test': 5219.836299709872, 'R2_Test': 0.8244280822823356, 'Adj_R2_Test': 0.8187166086913812}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9196.055248,6324.875376,0.745244,0.744625,6753.577811,5219.8363,0.824428,0.818717


**Results:**
- Before Polynominal Features: MAE: 4495.405879	R2: 0.860075
- After Polynominal Features: MAE: 5219.8363	R2: 0.824428

-> Polynominal Features did not improve the model and will not be used

## Skewness

In [428]:
#pd.set_option('display.max_rows', None)
num_columns_float = df_nans_handeled_cat.select_dtypes(include='number').columns
skew = df_nans_handeled_cat[num_columns_float].skew().sort_values(ascending=False)
skew[skew >= 3]

Assortment_b            10.995450
StoreType_b              7.912346
StateHoliday_c           7.611515
StateHoliday_b           5.003762
Customers_Lag_1_MA_8     3.079217
Customers_Lag_2_MA_8     3.073890
Customers_Lag_3_MA_8     3.068401
Customers_Lag_1_MA_6     3.065142
Customers_Lag_4_MA_8     3.063038
Customers_Lag_2_MA_6     3.060226
Customers_Lag_5_MA_8     3.059181
Customers_Lag_3_MA_6     3.055243
Customers_Lag_6_MA_8     3.054436
Customers_Lag_7_MA_8     3.050417
Customers_Lag_1_MA_4     3.049608
Customers_Lag_4_MA_6     3.049238
Customers_Lag_8_MA_8     3.046789
Customers_Lag_2_MA_4     3.046094
Customers_Lag_5_MA_6     3.044501
Customers_Lag_3_MA_4     3.040599
Customers_Lag_6_MA_6     3.038744
Customers_Lag_4_MA_4     3.034761
Customers_Lag_7_MA_6     3.034684
Customers_Lag_5_MA_4     3.031263
Customers_Lag_8_MA_6     3.030890
Customers_Lag_6_MA_4     3.024336
Customers_Lag_7_MA_4     3.018899
Customers_Lag_8_MA_4     3.014113
dtype: float64

In [429]:
testModelsTestSplit8W(df_nans_handeled_cat, None)


{'Model': 'LinearRegression', 'RMSE_Train': 9348.624262728845, 'MAE_Train': 6448.18666177155, 'R2_Train': 0.7367209231767318, 'Adj_R2_Train': 0.7362865518161379, 'RMSE_Test': 6029.131809472689, 'MAE_Test': 4495.405879127806, 'R2_Test': 0.8600745236580398, 'Adj_R2_Test': 0.8570124514786959}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9348.624263,6448.186662,0.736721,0.736287,6029.131809,4495.405879,0.860075,0.857012


In [430]:
# Log transformation
df_nans_handeled_cat_log = df_nans_handeled_cat.copy()
for_log_transform = skew[skew >= 3].index
df_nans_handeled_cat_log[for_log_transform] = np.log(df_nans_handeled_cat_log[for_log_transform]+1)
testModelsTestSplit8W(df_nans_handeled_cat_log, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9011.436940430183, 'MAE_Train': 6141.00339989779, 'R2_Train': 0.7553703847711067, 'Adj_R2_Train': 0.7549667822513368, 'RMSE_Test': 6074.646609651307, 'MAE_Test': 4512.055024525082, 'R2_Test': 0.8579539134781295, 'Adj_R2_Test': 0.8548454347286248}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9011.43694,6141.0034,0.75537,0.754967,6074.64661,4512.055025,0.857954,0.854845


In [431]:
# use PowerTransformer to transform the data
df_nans_handeled_cat_power = df_nans_handeled_cat.copy()
for_log_transform = skew[skew >= 3].index
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()
df_nans_handeled_cat_power[for_log_transform] = pt.fit_transform(df_nans_handeled_cat_power[for_log_transform])
testModelsTestSplit8W(df_nans_handeled_cat_power, None)


{'Model': 'LinearRegression', 'RMSE_Train': 9080.95770607921, 'MAE_Train': 6230.562682192093, 'R2_Train': 0.7515813243113636, 'Adj_R2_Train': 0.7511714704047873, 'RMSE_Test': 5959.227147139775, 'MAE_Test': 4414.332558622488, 'R2_Test': 0.8633004401241766, 'Adj_R2_Test': 0.860308962587939}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.957706,6230.562682,0.751581,0.751171,5959.227147,4414.332559,0.8633,0.860309


**Results:**
- Before: MAE:4793.96383	R2:0.844227
- After Power Transformation skew>=2: MAE:4615.821447	R2:	0.850871
- After Power Transformation skew>=3: MAE:4682.241185	R2: 0.847547
- After log transformation skew>=2: MAE: 4641.626858	R2: 0.851209
- After log transformation skew>=3: MAE: 4528.091135	R2: 0.854112

-> Log transformation with skew>=3 will be used

After adding ma3:
- Before: MAE: 4495.405879	R2: 0.860075
- After log transformation skew>=3: MAE: 4512.055025	R2:0.857954  
- After Power transformation skew>=3: MAE: 4414.332559	R2: 0.8633

--> Power transformation with skew>=3 will be used

## Feature Scaling

In [432]:
testModelsTestSplit8W(df_nans_handeled_cat_power, None)

{'Model': 'LinearRegression', 'RMSE_Train': 9080.95770607921, 'MAE_Train': 6230.562682192093, 'R2_Train': 0.7515813243113636, 'Adj_R2_Train': 0.7511714704047873, 'RMSE_Test': 5959.227147139775, 'MAE_Test': 4414.332558622488, 'R2_Test': 0.8633004401241766, 'Adj_R2_Test': 0.860308962587939}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9080.957706,6230.562682,0.751581,0.751171,5959.227147,4414.332559,0.8633,0.860309


### StandardScaler

In [433]:
testModelsTestSplit8W(df_nans_handeled_cat_power, StandardScaler())

{'Model': 'LinearRegression', 'RMSE_Train': 9081.040878116439, 'MAE_Train': 6229.594563260584, 'R2_Train': 0.7515767737820847, 'Adj_R2_Train': 0.7511669123678111, 'RMSE_Test': 5962.061855840987, 'MAE_Test': 4418.61090177654, 'R2_Test': 0.863170357619569, 'Adj_R2_Test': 0.8601760334107396}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9081.040878,6229.594563,0.751577,0.751167,5962.061856,4418.610902,0.86317,0.860176


### MinMaxScaler

In [434]:
testModelsTestSplit8W(df_nans_handeled_cat_power, MinMaxScaler())

{'Model': 'LinearRegression', 'RMSE_Train': 9081.085750405848, 'MAE_Train': 6227.987120558813, 'R2_Train': 0.751574318700868, 'Adj_R2_Train': 0.7511644532360752, 'RMSE_Test': 5941.910085649842, 'MAE_Test': 4398.551737668162, 'R2_Test': 0.8640937628566556, 'Adj_R2_Test': 0.8611196460722401}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9081.08575,6227.987121,0.751574,0.751164,5941.910086,4398.551738,0.864094,0.86112


### RobustScaler

In [400]:
testModelsTestSplit8W(df_nans_handeled_cat_power, RobustScaler())

{'Model': 'LinearRegression', 'RMSE_Train': 9081.139632074473, 'MAE_Train': 6228.870536176268, 'R2_Train': 0.7515713706767402, 'Adj_R2_Train': 0.7511615003481455, 'RMSE_Test': 5945.909267236711, 'MAE_Test': 4398.59384809417, 'R2_Test': 0.8639107588669345, 'Adj_R2_Test': 0.8609326372976843}


,Model,RMSE_Train,MAE_Train,R2_Train,Adj_R2_Train,RMSE_Test,MAE_Test,R2_Test,Adj_R2_Test
0,LinearRegression,9081.139632,6228.870536,0.751571,0.751162,5945.909267,4398.593848,0.863911,0.860933


**Result: StandardScaler, MinMaxScaler, RobustScaler have just a small effect on the model performance.**
- Before at LinearRegression: MAE: 4414.332559	R2: 0.8633
- After MinMaxScaler at LinearRegression: MAE: 4398.551738	R2: 0.864094

-> MinMaxScaler will be used

In [435]:
# Save df as csv
df_nans_handeled_cat_power.to_csv('df_nans_handeled_cat_power.csv', index=False)

TODO: Test Cyclical Feature Encoding

Cyclical encoding is a technique used to transform cyclical features like hours of the day, days of the week, months in a year, or calendar weeks, so that these features can be properly understood and utilized by machine learning models. The key challenge with cyclical data is that they wrap around: the highest value is succeeded by the lowest value, forming a cycle. Traditional numerical encoding of these values can mislead a model into thinking that the cycle is linear and that values at the end of the cycle are far apart when, in reality, they are close.

How Cyclical Encoding Works:
The most common way to encode cyclical data is by using sine and cosine transformations. This approach projects each cyclical feature onto a circle such that the beginning and end of the cycle meet. Here's how it's typically done:

Sine and Cosine Transformation: Each cyclical value is transformed into two features using sine and cosine functions. This maps the data points onto a unit circle and preserves their cyclical nature.

Example: For a month feature (with values from 1 to 12), the transformation would be:

python
Copy code
df['month_sin'] = np.sin((df['month']-1)*(2.*np.pi/12))
df['month_cos'] = np.cos((df['month']-1)*(2.*np.pi/12))
This creates two new columns, month_sin and month_cos, which represent the sine and cosine values of the month on the unit circle.

Advantages of Cyclical Encoding:
Preserves Cyclical Nature: It correctly represents the proximity of values at the end and start of the cycle. For example, December (12) is close to January (1).

Continuous Representation: It provides a smooth and continuous representation, which is useful for many machine learning models.

Efficient Use of Features: Unlike one-hot encoding, which would create many sparse columns for each possible value, sine and cosine transformations keep the feature space more compact.

When to Use Cyclical Encoding:
Cyclical encoding is particularly useful in time series analysis or when the cyclical aspect of a feature is important for the model. Common scenarios include:

Time of Day: Hours of the day for predicting traffic patterns.
Days of the Week: Days for weekly sales forecasts.
Seasons: Months or calendar weeks for seasonal effects in data.
Implementation Considerations:
Standardization: After applying sine and cosine transformations, you might need to standardize these features, especially if you're using models sensitive to feature scaling.

Model Compatibility: While cyclical encoding is beneficial for many models, it's always good practice to test and validate its impact on your specific model and dataset.

In summary, cyclical encoding is a powerful technique for handling cyclical features in machine learning, enabling models to better understand and utilize the patterns within such data.